In [17]:
import torch
import numpy as np

def Energy_loc(tot_num, energy): 
    loc = torch.round(tot_num / 150 * energy).long()  # Converted to long for indexing
    return loc

NumOneHot = 300
path = 'MgO/' # 'around_dos_tensor.pth',

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(path)
dpath = path + 'DataSet/'
energy = torch.from_numpy(np.loadtxt(dpath + 'EnergyFull')).to(device)
nband = energy.shape[1]

# print(energy[0].detach(),nband)
energy = torch.flatten(energy.t()).view(len(energy)*nband,1)
# print(energy[0].detach())

QP = torch.from_numpy(np.loadtxt(dpath + 'QPFull')).to(device)
QP = QP.repeat(nband,1)
Tau = torch.from_numpy(np.loadtxt(dpath + 'TauFull')).to(device)
Tau = torch.flatten(Tau.t()).view(len(Tau)*nband,1)
vx = torch.from_numpy(np.loadtxt(dpath + 'VXFull')).to(device)
vx = vx[:,1:]
vx = torch.flatten(vx.t()).view(len(vx)*nband,1)
vx = vx.pow(2)
print(QP.shape, energy.shape, vx.shape, Tau.shape)
all_data = torch.cat((QP, energy, vx, Tau), 1)
nkpt = all_data.size(0)
print('all_data.shape',all_data.shape)
print('Data:Qpoints, energy, square velocity, Tau')
print(nkpt)
na, nb,nc,ne = 30,30,30, NumOneHot 
One_hot = torch.zeros((na,nb,nc,ne), device=device)
print(One_hot.shape)
oh_data = all_data[:,0:4].clone()
oh_data[:,0:3] = oh_data[:,0:3] / 0.033333
oh_data[:,3] = Energy_loc(NumOneHot,oh_data[:,3])
oh_data = oh_data.type(torch.int)
print('max energy',torch.max(oh_data[:,3]))
for qp in range(nkpt):
    One_hot[oh_data[qp][0],oh_data[qp][1],oh_data[qp][2],oh_data[qp][3]] += 1



MgO/
torch.Size([162000, 3]) torch.Size([162000, 1]) torch.Size([162000, 1]) torch.Size([162000, 1])
all_data.shape torch.Size([162000, 6])
Data:Qpoints, energy, square velocity, Tau
162000
torch.Size([30, 30, 30, 300])
max energy tensor(252, device='cuda:0', dtype=torch.int32)


In [32]:
# Weight:
Weight = torch.ones(nkpt)
print(all_data[3,:].detach())
print(torch.where(all_data[:,3] == 0))
indice = torch.where(all_data[:,3] == 0)
Weight[indice] = 0
print(torch.where(Weight == 0))
# print(torch.where[all_data[:,0:3] == torch.tensor([0,0,0],device=device)])
loc = torch.where((all_data[:,0] == 0) & (all_data[:,1] == 0) & (all_data[:,2] == 0))
print('loc0',loc[0][3])
lowOptic = all_data[loc[0][3]][3]
print('low Optic',lowOptic)
indice = torch.where(all_data[:,3] <= lowOptic)
Weight[indice] = 10
print(Weight.__len__())
print(Weight)


tensor([ 0.1000,  0.0000,  0.0000, 13.5891,  9.0579,  8.8887], device='cuda:0',
       dtype=torch.float64)
(tensor([    0, 27000, 54000], device='cuda:0'),)
(tensor([    0, 27000, 54000]),)
loc0 tensor(81000, device='cuda:0')
low Optic tensor(69.6717, device='cuda:0', dtype=torch.float64)
162000
tensor([10., 10., 10.,  ...,  1.,  1.,  1.])
